Может понадобиться установка:

In [92]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Импортируем библиотеки:

In [93]:
import os.path
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd
import numpy as np
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials

Забираем данные, используя Google Sheeets API v4:

In [94]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SERVICE_ACCOUNT_FILE = r"/content/drive/MyDrive/some_data/some-data-315613-ac85ae5ff65d.json"
SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

In [95]:
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [96]:
service = build("sheets", "v4", credentials=credentials)

Выбираем страницы для дальнейшей обработки раздельно:

In [97]:
result_transactions = service.spreadsheets().values().get(
    spreadsheetId=SAMPLE_SPREADSHEET_ID,
    range='transactions'
).execute()

result_clients = service.spreadsheets().values().get(
    spreadsheetId=SAMPLE_SPREADSHEET_ID,
    range='clients'
).execute()

result_managers = service.spreadsheets().values().get(
    spreadsheetId=SAMPLE_SPREADSHEET_ID,
    range='managers'
).execute()

result_leads = service.spreadsheets().values().get(
    spreadsheetId=SAMPLE_SPREADSHEET_ID,
    range='leads'
).execute()

Посомтрим данные на вид:

In [98]:
print(result_managers.keys())

dict_keys(['range', 'majorDimension', 'values'])


In [99]:
print(result_managers["majorDimension"])

ROWS


In [100]:
print(result_managers["values"])

[['manager_id', 'd_manager', 'd_club'], ['1e9c5562-0cfc-11ea-abe1-c412f533dba1', 'manager #8', 'club #2'], ['43756fa4-57a0-11e9-ab9a-c412f533dba1', 'manager #11', 'club #4'], ['543888c4-fbb3-11e9-abdc-c412f533dba1', 'manager #12', 'club #3'], ['5dd88f32-6501-11ea-abf3-c412f533dba1', 'manager #10', 'club #2'], ['9a87c608-52dc-11ea-abeb-c412f533dba1', 'manager #9', 'club #3'], ['abed5496-ca88-11e9-abb8-c412f533dba1', 'manager #5', 'club #4'], ['ad52c7a8-a752-11e7-8115-c412f533dba1', 'manager #7', 'club #1'], ['b55a233c-5df6-11ea-abf2-c412f533dba1', 'manager #2', 'club #4'], ['b832aac3-fcda-11ea-ac18-c412f533dba1', 'manager #13', 'club #3'], ['d12fe765-b444-11e9-abb3-c412f533dba1', 'manager #1', 'club #1'], ['db06b8bc-bfb2-11e7-8116-c412f533dba1', 'manager #4', 'club #2'], ['e463b03f-2d74-11eb-ac1f-c412f533dba1', 'manager #3', 'club #1'], ['f44596c6-12cd-11eb-ac1c-c412f533dba1', 'manager #6', 'club #2']]


Придаём таблицам рабочий вид и делаем предпросмотр:

In [101]:
df_transactions = pd.DataFrame(result_transactions["values"][1:], columns=result_transactions["values"][0])

In [102]:
df_transactions['created_at'] = pd.to_datetime(df_transactions['created_at'])

Нужно переименовать столбец киентов, иначе пандас не осилит в конце (обратить внимание, что lead_client_id не относится к таблице df_leads):

In [103]:
df_transactions["lead_client_id"] = df_transactions["l_client_id"]

In [104]:
df_transactions = df_transactions.drop(["l_client_id"], axis=1)

In [105]:
df_transactions

,transaction_id,created_at,m_real_amount,lead_client_id
0,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,31,8a805e60-6fd6-11e7-80fc-c412f533dba1
1,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,87,8a805e60-6fd6-11e7-80fc-c412f533dba1
2,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,49231,dc0d0e52-629a-11ea-abf3-c412f533dba1
3,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,17305,dc0d0e52-629a-11ea-abf3-c412f533dba1
4,52f8ebcc-d82d-4be1-8004-72cfbe0dff24,2020-09-25 06:00:07,9022,e663b6a0-4a1a-11e8-a2c3-c412f533dba1
...,...,...,...,...
28995,39119f9c-d15b-46e6-bffa-1361257415cd,2020-07-21 21:02:34,0,b56326a0-d5a2-11e9-abbe-c412f533dba1
28996,74fa1409-2a93-4a3e-bffb-d2cd0c75f6a5,2020-01-23 03:45:41,0,1a32cb98-d5a2-11e9-abbe-c412f533dba1
28997,b5959542-894b-4eda-bffc-8ecaaed4f9e5,2020-09-03 06:45:51,0,55953b37-d5af-11e9-abbe-c412f533dba1
28998,c7cb1253-6e03-4531-bffe-81866395abce,2020-08-13 22:42:08,0,547f3c7b-d5a4-11e9-abbe-c412f533dba1


In [106]:
df_transactions.dtypes

transaction_id            object
created_at        datetime64[ns]
m_real_amount             object
lead_client_id            object
dtype: object

In [107]:
df_clients = pd.DataFrame(result_clients["values"][1:], columns=result_clients["values"][0])

In [108]:
df_clients.shape

(75766, 3)

In [109]:
df_clients

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1
...,...,...,...
75761,ffeed40c-3a7e-11e9-ab8e-c412f533dba1,2019-02-27 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
75762,fff022a5-2c34-11e8-8131-c412f533dba1,2018-03-20 00:00:00,d12fe765-b444-11e9-abb3-c412f533dba1
75763,fff88086-11bb-11ea-abe7-c412f533dba1,2019-11-28 12:48:58,bf340611-034b-11eb-ac18-c412f533dba1
75764,fff9ba6e-de05-11e9-abc1-c412f533dba1,2019-09-23 17:27:42,43756fa4-57a0-11e9-ab9a-c412f533dba1


Заменим поломанные даты на воспринимаемые программой:

In [110]:
df_clients.loc[(df_clients['created_at'] < "0001-01-02 00:00:00"), 'created_at'] = '2001-01-01 00:00:01'

In [111]:
df_clients

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1
...,...,...,...
75761,ffeed40c-3a7e-11e9-ab8e-c412f533dba1,2019-02-27 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
75762,fff022a5-2c34-11e8-8131-c412f533dba1,2018-03-20 00:00:00,d12fe765-b444-11e9-abb3-c412f533dba1
75763,fff88086-11bb-11ea-abe7-c412f533dba1,2019-11-28 12:48:58,bf340611-034b-11eb-ac18-c412f533dba1
75764,fff9ba6e-de05-11e9-abc1-c412f533dba1,2019-09-23 17:27:42,43756fa4-57a0-11e9-ab9a-c412f533dba1


Поменяем тип данных в столбце created_at на date:

In [112]:
df_clients['created_at'] = pd.to_datetime(df_clients['created_at'])

Удалим из этого списка даты, после первой записи в таблице df_leads для дальнейшего выявления старых / новых пользователей:

In [113]:
df_clients = df_clients.query("created_at < '2020-10-01 0:02:55'")

In [114]:
df_clients

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1
...,...,...,...
75761,ffeed40c-3a7e-11e9-ab8e-c412f533dba1,2019-02-27 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
75762,fff022a5-2c34-11e8-8131-c412f533dba1,2018-03-20 00:00:00,d12fe765-b444-11e9-abb3-c412f533dba1
75763,fff88086-11bb-11ea-abe7-c412f533dba1,2019-11-28 12:48:58,bf340611-034b-11eb-ac18-c412f533dba1
75764,fff9ba6e-de05-11e9-abc1-c412f533dba1,2019-09-23 17:27:42,43756fa4-57a0-11e9-ab9a-c412f533dba1


In [115]:
df_clients.dtypes

client_id               object
created_at      datetime64[ns]
l_manager_id            object
dtype: object

In [116]:
df_managers = pd.DataFrame(result_managers["values"][1:], columns=result_managers["values"][0])

In [117]:
df_managers.shape

(13, 3)

In [118]:
df_managers

,manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,43756fa4-57a0-11e9-ab9a-c412f533dba1,manager #11,club #4
2,543888c4-fbb3-11e9-abdc-c412f533dba1,manager #12,club #3
3,5dd88f32-6501-11ea-abf3-c412f533dba1,manager #10,club #2
4,9a87c608-52dc-11ea-abeb-c412f533dba1,manager #9,club #3
5,abed5496-ca88-11e9-abb8-c412f533dba1,manager #5,club #4
6,ad52c7a8-a752-11e7-8115-c412f533dba1,manager #7,club #1
7,b55a233c-5df6-11ea-abf2-c412f533dba1,manager #2,club #4
8,b832aac3-fcda-11ea-ac18-c412f533dba1,manager #13,club #3
9,d12fe765-b444-11e9-abb3-c412f533dba1,manager #1,club #1


In [119]:
df_leads = pd.DataFrame(result_leads["values"][1:], columns=result_leads["values"][0])

In [120]:
df_leads['created_at'] = pd.to_datetime(df_leads['created_at'])

In [121]:
df_leads.shape

(3337, 6)

In [122]:
df_leads.dtypes

lead_id                 object
created_at      datetime64[ns]
d_utm_medium            object
d_utm_source            object
l_manager_id            object
l_client_id             object
dtype: object

In [123]:
df_leads

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1
...,...,...,...,...,...,...
3332,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1
3333,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vk,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1
3334,ffbd53f1-3150-11eb-ac1f-c412f533dba1,2020-11-28 12:09:13,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,6dd39f15-d5a3-11e9-abbe-c412f533dba1
3335,ffc0b363-2ae9-11eb-ac1f-c412f533dba1,2020-11-20 08:36:48,cpc,instagram,ad52c7a8-a752-11e7-8115-c412f533dba1,00000000-0000-0000-0000-000000000000


________

Все 4 таблицы подготовлены, начинаем соединять:
________

In [124]:
# df_transactions
# df_clients
# df_managers
# df_leads

In [125]:
df_clients.shape

(74017, 3)

In [126]:
df_leads.shape

(3337, 6)

Соединим **df_clients** и **df_leads**:

In [127]:
df_cl = df_clients.merge(df_leads, left_on="client_id", right_on="l_client_id", how="outer", sort=False)

In [128]:
df_cl

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
76538,NaN,NaT,NaN,fee0af4b-0a44-11eb-ac1a-c412f533dba1,2020-10-09 19:35:02,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,f3268586-0a45-11eb-ac1a-c412f533dba1
76539,NaN,NaT,NaN,ff230ba3-0912-11eb-ac1a-c412f533dba1,2020-10-08 07:04:36,cpc,yandex,9a87c608-52dc-11ea-abeb-c412f533dba1,78d56291-093f-11eb-ac1a-c412f533dba1
76540,NaN,NaT,NaN,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1
76541,NaN,NaT,NaN,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1


In [58]:
# df_cl.to_excel("/content/drive/MyDrive/some_data/df_cl.xlsx")

____________

Присоединяем df_managers по ключам менеджеров:

In [129]:
df_clm = df_cl.merge(df_managers, left_on="l_manager_id_y", right_on="manager_id", how="outer", sort=False)

In [130]:
df_clm

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
76539,NaN,NaT,NaN,7b16ce91-25cb-11eb-ac1d-c412f533dba1,2020-11-13 20:15:44,,,994accfd-03e6-11eb-ac18-c412f533dba1,5553ddfc-27d1-11eb-ac1f-c412f533dba1,NaN,NaN,NaN
76540,NaN,NaT,NaN,7dd69b9a-0d24-11eb-ac1a-c412f533dba1,2020-10-13 11:19:55,,,31298620-0355-11eb-ac18-c412f533dba1,76d55455-0d24-11eb-ac1a-c412f533dba1,NaN,NaN,NaN
76541,NaN,NaT,NaN,88f7896c-1a92-11eb-ac1d-c412f533dba1,2020-10-30 13:30:24,,,31d3f905-0c7c-11eb-ac1a-c412f533dba1,88f7896d-1a92-11eb-ac1d-c412f533dba1,NaN,NaN,NaN
76542,NaN,NaT,NaN,f9be181c-0d10-11eb-ac1a-c412f533dba1,2020-10-13 09:00:13,cpc,google,d6a49e22-f74f-11ea-ac18-c412f533dba1,a601b931-0d38-11eb-ac1a-c412f533dba1,NaN,NaN,NaN


Удалим строку с менеджером без продаж.

In [131]:
df_clm = df_clm.drop([76543], axis=0)

In [132]:
df_clm

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
76538,NaN,NaT,NaN,623dbf3c-2053-11eb-ac1d-c412f533dba1,2020-11-06 21:13:27,,,997dc1c9-a85e-11e9-abb0-c412f533dba1,623dbf3d-2053-11eb-ac1d-c412f533dba1,NaN,NaN,NaN
76539,NaN,NaT,NaN,7b16ce91-25cb-11eb-ac1d-c412f533dba1,2020-11-13 20:15:44,,,994accfd-03e6-11eb-ac18-c412f533dba1,5553ddfc-27d1-11eb-ac1f-c412f533dba1,NaN,NaN,NaN
76540,NaN,NaT,NaN,7dd69b9a-0d24-11eb-ac1a-c412f533dba1,2020-10-13 11:19:55,,,31298620-0355-11eb-ac18-c412f533dba1,76d55455-0d24-11eb-ac1a-c412f533dba1,NaN,NaN,NaN
76541,NaN,NaT,NaN,88f7896c-1a92-11eb-ac1d-c412f533dba1,2020-10-30 13:30:24,,,31d3f905-0c7c-11eb-ac1a-c412f533dba1,88f7896d-1a92-11eb-ac1d-c412f533dba1,NaN,NaN,NaN


__________

Добавляем df_transactions:

In [133]:
df_clmt = df_clm.merge(df_transactions, left_on="l_client_id", right_on="lead_client_id", how="outer", sort=False)

In [134]:
df_clmt['m_real_amount'] = pd.to_numeric(df_clmt['m_real_amount'])

In [135]:
df_clmt

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club,transaction_id,created_at,m_real_amount,lead_client_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104817,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e22d08ce-11bf-41a2-be68-3308332f42a6,2020-01-07 08:00:59,0.0,befc3c0b-d5a3-11e9-abbe-c412f533dba1
104818,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96a35b77-b3af-4d3b-be7b-2001cf8fb4f7,2020-02-07 03:45:45,0.0,eef0ac0f-e919-11e9-abd7-c412f533dba1
104819,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,834b104c-b0d6-4dcd-bee4-20b9ce8dcae7,2020-01-23 03:45:34,0.0,7bac29e7-d5a7-11e9-abbe-c412f533dba1
104820,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c64c03a7-b5b4-49f4-bf5f-268227e105ee,2020-01-23 03:45:36,0.0,760e1f4e-6fd8-11e7-80fc-c412f533dba1


In [136]:
df_clmt.dtypes

client_id                 object
created_at_x      datetime64[ns]
l_manager_id_x            object
lead_id                   object
created_at_y      datetime64[ns]
d_utm_medium              object
d_utm_source              object
l_manager_id_y            object
l_client_id               object
manager_id                object
d_manager                 object
d_club                    object
transaction_id            object
created_at        datetime64[ns]
m_real_amount            float64
lead_client_id            object
dtype: object

In [137]:
# df_clmt.to_excel("/content/drive/MyDrive/some_data/df_clmt_ready.xlsx")

__________
Соединили таблицы, теперь начинаем работать.
________

Создадим столбцы с пометками о проверке на дроп и на старого клиента:

In [138]:
df_clmt["dropped"] = df_clmt.l_client_id == "00000000-0000-0000-0000-000000000000"

In [139]:
df_clmt["old"] = df_clmt.l_client_id == df_clmt.client_id

In [140]:
df_clmt

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club,transaction_id,created_at,m_real_amount,lead_client_id,dropped,old
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104817,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e22d08ce-11bf-41a2-be68-3308332f42a6,2020-01-07 08:00:59,0.0,befc3c0b-d5a3-11e9-abbe-c412f533dba1,False,False
104818,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96a35b77-b3af-4d3b-be7b-2001cf8fb4f7,2020-02-07 03:45:45,0.0,eef0ac0f-e919-11e9-abd7-c412f533dba1,False,False
104819,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,834b104c-b0d6-4dcd-bee4-20b9ce8dcae7,2020-01-23 03:45:34,0.0,7bac29e7-d5a7-11e9-abbe-c412f533dba1,False,False
104820,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c64c03a7-b5b4-49f4-bf5f-268227e105ee,2020-01-23 03:45:36,0.0,760e1f4e-6fd8-11e7-80fc-c412f533dba1,False,False


Создадим столбец с замером разницы между оплатой и временем регистрации заказа:

In [141]:
df_clmt["fast_buy"] = df_clmt["created_at_y"] - df_clmt["created_at"]

In [142]:
df_clmt["f_buy"] = df_clmt.fast_buy.dt.days

In [143]:
df_clmt

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club,transaction_id,created_at,m_real_amount,lead_client_id,dropped,old,fast_buy,f_buy
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104817,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e22d08ce-11bf-41a2-be68-3308332f42a6,2020-01-07 08:00:59,0.0,befc3c0b-d5a3-11e9-abbe-c412f533dba1,False,False,NaT,NaN
104818,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96a35b77-b3af-4d3b-be7b-2001cf8fb4f7,2020-02-07 03:45:45,0.0,eef0ac0f-e919-11e9-abd7-c412f533dba1,False,False,NaT,NaN
104819,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,834b104c-b0d6-4dcd-bee4-20b9ce8dcae7,2020-01-23 03:45:34,0.0,7bac29e7-d5a7-11e9-abbe-c412f533dba1,False,False,NaT,NaN
104820,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c64c03a7-b5b4-49f4-bf5f-268227e105ee,2020-01-23 03:45:36,0.0,760e1f4e-6fd8-11e7-80fc-c412f533dba1,False,False,NaT,NaN


In [144]:
# старых клиентов 1667
df_clmt["old"].value_counts()

False    103155
True       1667
Name: old, dtype: int64

In [145]:
# дропнутых клиентов 824
df_clmt["dropped"].value_counts()

False    103998
True        824
Name: dropped, dtype: int64

In [146]:
df_clmt["is_fast_buy"] = (df_clmt["f_buy"] >= 0) & (df_clmt["f_buy"] <= 7)

Удаляем все строки, где нет ID клиента в заказах (дропнутные пользователи остаются в списке):

In [147]:
df_clmt = df_clmt[df_clmt["l_client_id"].notna()]

In [148]:
df_clmt

,client_id,created_at_x,l_manager_id_x,lead_id,created_at_y,d_utm_medium,d_utm_source,l_manager_id_y,l_client_id,manager_id,d_manager,d_club,transaction_id,created_at,m_real_amount,lead_client_id,dropped,old,fast_buy,f_buy,is_fast_buy
73206,00817845-6fd6-11e7-80fc-c412f533dba1,2016-08-23 00:00:00,1e9c5562-0cfc-11ea-abe1-c412f533dba1,b3a85c94-1d2c-11eb-ac1d-c412f533dba1,2020-11-02 20:59:00,cpc,instagram,1e9c5562-0cfc-11ea-abe1-c412f533dba1,00817845-6fd6-11e7-80fc-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,NaT,NaN,False
73207,05b7faaa-d149-11e9-abbd-c412f533dba1,2019-09-07 12:25:00,8e5a4a2a-02ef-11eb-ac18-c412f533dba1,0f5f7b15-2d6e-11eb-ac1f-c412f533dba1,2020-11-23 13:27:10,cpi,sms,1e9c5562-0cfc-11ea-abe1-c412f533dba1,05b7faaa-d149-11e9-abbd-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,NaT,NaN,False
73208,0ed85394-c4da-11e9-abb6-c412f533dba1,2019-08-22 00:00:00,5dd88f32-6501-11ea-abf3-c412f533dba1,1be590c9-0e2b-11eb-ac1a-c412f533dba1,2020-10-14 18:39:54,,,1e9c5562-0cfc-11ea-abe1-c412f533dba1,0ed85394-c4da-11e9-abb6-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,NaT,NaN,False
73209,10530fbf-ed99-11e9-abda-c412f533dba1,2019-10-13 13:08:25,1e9c5562-0cfc-11ea-abe1-c412f533dba1,202ea177-0ecf-11eb-ac1a-c412f533dba1,2020-10-15 14:13:53,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,10530fbf-ed99-11e9-abda-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2,849602e9-c7cf-47ce-bc03-6d12f651dfff,2020-10-18 10:33:37,22604.0,10530fbf-ed99-11e9-abda-c412f533dba1,False,True,-3 days +03:40:16,-3.0,False
73210,126f95f3-6fd6-11e7-80fc-c412f533dba1,2016-01-19 00:00:00,5dd88f32-6501-11ea-abf3-c412f533dba1,4bc28390-1889-11eb-ac1d-c412f533dba1,2020-10-27 23:19:13,cpc,instagram,1e9c5562-0cfc-11ea-abe1-c412f533dba1,126f95f3-6fd6-11e7-80fc-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,NaT,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77496,NaN,NaT,NaN,623dbf3c-2053-11eb-ac1d-c412f533dba1,2020-11-06 21:13:27,,,997dc1c9-a85e-11e9-abb0-c412f533dba1,623dbf3d-2053-11eb-ac1d-c412f533dba1,NaN,NaN,NaN,9c146fa2-241d-4f61-b53a-7b1636015c3b,2020-11-06 18:19:42,39802.0,623dbf3d-2053-11eb-ac1d-c412f533dba1,False,False,0 days 02:53:45,0.0,True
77497,NaN,NaT,NaN,7b16ce91-25cb-11eb-ac1d-c412f533dba1,2020-11-13 20:15:44,,,994accfd-03e6-11eb-ac18-c412f533dba1,5553ddfc-27d1-11eb-ac1f-c412f533dba1,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN,False
77498,NaN,NaT,NaN,7dd69b9a-0d24-11eb-ac1a-c412f533dba1,2020-10-13 11:19:55,,,31298620-0355-11eb-ac18-c412f533dba1,76d55455-0d24-11eb-ac1a-c412f533dba1,NaN,NaN,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaN,False
77499,NaN,NaT,NaN,88f7896c-1a92-11eb-ac1d-c412f533dba1,2020-10-30 13:30:24,,,31d3f905-0c7c-11eb-ac1a-c412f533dba1,88f7896d-1a92-11eb-ac1d-c412f533dba1,NaN,NaN,NaN,8988b78b-b820-4d98-bfe8-d78502fa4f71,2020-10-30 10:31:05,2133.0,88f7896d-1a92-11eb-ac1d-c412f533dba1,False,False,0 days 02:59:19,0.0,True


Удалим лишние столбцы, которые более не понадобятся:

In [149]:
df_clmt = df_clmt.drop(["client_id", "created_at_x", "l_manager_id_x", "d_utm_medium", "l_manager_id_y", "manager_id", "fast_buy", "f_buy"], axis=1)

In [150]:
df_clmt

,lead_id,created_at_y,d_utm_source,l_client_id,d_manager,d_club,transaction_id,created_at,m_real_amount,lead_client_id,dropped,old,is_fast_buy
73206,b3a85c94-1d2c-11eb-ac1d-c412f533dba1,2020-11-02 20:59:00,instagram,00817845-6fd6-11e7-80fc-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,False
73207,0f5f7b15-2d6e-11eb-ac1f-c412f533dba1,2020-11-23 13:27:10,sms,05b7faaa-d149-11e9-abbd-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,False
73208,1be590c9-0e2b-11eb-ac1a-c412f533dba1,2020-10-14 18:39:54,,0ed85394-c4da-11e9-abb6-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,False
73209,202ea177-0ecf-11eb-ac1a-c412f533dba1,2020-10-15 14:13:53,viber,10530fbf-ed99-11e9-abda-c412f533dba1,manager #8,club #2,849602e9-c7cf-47ce-bc03-6d12f651dfff,2020-10-18 10:33:37,22604.0,10530fbf-ed99-11e9-abda-c412f533dba1,False,True,False
73210,4bc28390-1889-11eb-ac1d-c412f533dba1,2020-10-27 23:19:13,instagram,126f95f3-6fd6-11e7-80fc-c412f533dba1,manager #8,club #2,NaN,NaT,NaN,NaN,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77496,623dbf3c-2053-11eb-ac1d-c412f533dba1,2020-11-06 21:13:27,,623dbf3d-2053-11eb-ac1d-c412f533dba1,NaN,NaN,9c146fa2-241d-4f61-b53a-7b1636015c3b,2020-11-06 18:19:42,39802.0,623dbf3d-2053-11eb-ac1d-c412f533dba1,False,False,True
77497,7b16ce91-25cb-11eb-ac1d-c412f533dba1,2020-11-13 20:15:44,,5553ddfc-27d1-11eb-ac1f-c412f533dba1,NaN,NaN,NaN,NaT,NaN,NaN,False,False,False
77498,7dd69b9a-0d24-11eb-ac1a-c412f533dba1,2020-10-13 11:19:55,,76d55455-0d24-11eb-ac1a-c412f533dba1,NaN,NaN,NaN,NaT,NaN,NaN,False,False,False
77499,88f7896c-1a92-11eb-ac1d-c412f533dba1,2020-10-30 13:30:24,,88f7896d-1a92-11eb-ac1d-c412f533dba1,NaN,NaN,8988b78b-b820-4d98-bfe8-d78502fa4f71,2020-10-30 10:31:05,2133.0,88f7896d-1a92-11eb-ac1d-c412f533dba1,False,False,True


Скачаем файл, для дальнейшей обработки в Табло и построения Дашборда:

In [151]:
# df_clmt.to_excel("/content/drive/MyDrive/some_data/df_ready.xlsx")